In [ ]:
#6ai) ADAM SGD OLS  

In [ ]:
# Importing various packages
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import sys
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from math import exp, sqrt
from random import random, seed

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#grid size
n = 200 

# Make data set.
x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05)) 

#polynomial fit
degree=6
poly = PolynomialFeatures(degree=degree)
X = poly.fit_transform(x)

# Hessian matrix
H = (2.0/n)* X.T @ X
invH = np.linalg.pinv(H)

# Get the eigenvalues
EigValues, EigVectors = np.linalg.eig(H)

n_epochs = 100000

nsizes= 4
sizes=[1, 2, 4, 8]

ngammas = 4 
gammas = np.logspace(-3, -1, ngammas)

beta_OLS_array = np.zeros((nsizes, ngammas), dtype=object)
y_OLS_SGD_adam_array = np.zeros((nsizes, ngammas), dtype=object)
MSE_OLS_SGD_adam = np.zeros((nsizes, ngammas))
    
for s in range(nsizes):
    M=sizes[s] #we vary the size of the minibatches
    m = int(n/M) #number of minibatches
    
    for g in range(ngammas):
        gamma = gammas[g]

        beta_OLS = np.random.randn(degree+1,1)

        # Value for parameter rho_1
        rho_1 = 0.9
        # Value for parameter rho_2
        rho_2 = 0.999
        # Including AdaGrad parameter to avoid possible division by zero
        delta  = 1e-8

        for epoch in range(n_epochs):
            Giter_1 = np.zeros(shape=(degree+1,1))
            Giter_2 = np.zeros(shape=(degree+1,1))

            for i in range(m):
                random_index = M*np.random.randint(m)
                xi = X[random_index:random_index+M]
                yi = y_noisy[random_index:random_index+M]

                gradient_OLS = (2.0/M)* xi.T @ ((xi @ beta_OLS)-yi)
                #print(np.shape(gradient_OLS))

                Previous_1= Giter_1
                #print(np.shape(Previous_1))

                Giter_1= gradient_OLS

                #update 1st momentum:        
                Gnew_1 = rho_1*Previous_1+(1-rho_1)*gradient_OLS
                #print(np.shape(Gnew_1))

                # Previous value for the outer product of gradients
                Previous_2 = Giter_2
                
                # Accumulated gradient
                Giter_2 += np.multiply(gradient_OLS, gradient_OLS)# @ gradient_OLS.T

                #update 2nd momentum:
                Gnew_2 = rho_2*Previous_2+(1-rho_2)*Giter_2

                #correction of 1st momentum bias:
                Gnew_1_bias= Gnew_1/(1.0-rho_1**epoch+1)

                #correction of 2nd momentum bias:
                Gnew_2_bias= Gnew_2/(1.0-rho_2**epoch+1)        

                # Hadamard product
                update = np.c_[gamma*Gnew_1_bias/(delta+np.sqrt(np.diagonal(Gnew_2_bias)))]

                beta_OLS -= update
                
        #storing every β for each M, γ combination
        beta_OLS_array[s, g] =beta_OLS
        
        y_OLS_SGD_adam = X @ beta_OLS
        
        y_OLS_SGD_adam_array[s, g] = y_OLS_SGD_adam

        MSE_OLS_SGD_adam[s, g] = MSE(y_OLS_SGD_adam, y_noisy)
        
#finding the minimum value of the MSE
MSE_OLS_SGD_adam_optimal = np.min(MSE_OLS_SGD_adam)
conditon = (MSE_OLS_SGD_adam == MSE_OLS_SGD_adam_optimal)
#the l, g for which we have the minimum MSE
result = np.where(conditon)
print(result)

print('method MSE=', MSE_OLS_SGD_adam_optimal)  

#saving the y that gives the optimal MSE
y_OLS_SGD_adam_optimal= y_OLS_SGD_adam_array[result]
y_OLS_SGD_adam_optimal=(y_OLS_SGD_adam_optimal[0])

#printing the final β 
beta_OLS_optimal = beta_OLS_array[result]
print(beta_OLS_optimal)
        
import pandas as pd
import seaborn as sns

pd.DataFrame(MSE_OLS_SGD_adam)
mse_data_ols_sgd_adam = pd.DataFrame(MSE_OLS_SGD_adam)

plt.figure(figsize=(12,7))
sns.heatmap(data=mse_data_ols_sgd_adam, annot=True,  fmt=".1e", cmap="crest")
plt.xlabel("initial γ")
plt.ylabel("number of minibatches")
plt.show()

plt.figure()
plt.plot(x*180/np.pi, y_noisy, 'ro', label='data')
plt.plot(x*180/np.pi, y_OLS_SGD_adam_optimal, label='ADAM SGD OLS')
plt.xlabel('latitude [degrees]')
plt.ylabel('f')
plt.legend()
plt.show()

#Calculating the time processing time
import time
start_time = time.time()
print("execution time=", (time.time() - start_time), 's')

In [ ]:
#6aii) ADAM SGD Ridge

In [ ]:
#Ridge parameter
nlambdas = 6 
lambdas = np.logspace(-5, 0, nlambdas)

ngammas = 6 
gammas = np.logspace(-3, 1, ngammas)

n_epochs = 10000

for M in [1, 2, 4]:  
    m = int(n/M) #number of minibatches

    beta_Ridge = np.random.randn(degree+1,1)
    
    y_Ridge_SGD_adam_array = np.zeros((nlambdas, ngammas), dtype=object)
    MSE_Ridge_SGD_adam = np.zeros((nlambdas, ngammas))

    # Value for parameter rho_1
    rho_1 = 0.9
    # Value for parameter rho_2
    rho_2 = 0.999
    
    # Including AdaGrad parameter to avoid possible division by zero
    delta  = 1e-8

    for l in range(nlambdas):
        lmbda = lambdas[l]
        beta_Ridge = np.random.randn(degree+1,1)    
                
        # We vary the learning rate
        for g in range(ngammas):
            gamma=gammas[g] 

            for epoch in range(n_epochs):
                Giter_1 = np.zeros(shape=(degree+1,1))    
                Giter_2 = np.zeros(shape=(degree+1,1))

                for i in range(m):
                    random_index = M*np.random.randint(m)
                    xi = X[random_index:random_index+M]
                    yi = y_noisy[random_index:random_index+M]

                    gradient_Ridge = (2.0/M)*xi.T @ (xi @ (beta_Ridge)-yi)+2*lmbda*beta_Ridge

                    Previous_1= Giter_1
                    #print(np.shape(Previous_1))

                    Giter_1= gradient_Ridge

                    #update 1st momentum:        
                    Gnew_1 = rho_1*Previous_1+(1-rho_1)*gradient_Ridge
                    #print(np.shape(Gnew_1))

                    # Previous value for the outer product of gradients
                    Previous_2 = Giter_2

                    # Accumulated gradient
                    Giter_2 += np.multiply(gradient_Ridge, gradient_Ridge)# @ gradient_OLS.T

                    #update 2nd momentum:
                    Gnew_2 = rho_2*Previous_2+(1-rho_2)*Giter_2

                    #correction of 1st momentum bias:
                    Gnew_1_bias= Gnew_1/(1.0-rho_1**epoch+1)#<-or maybe epochs??see the video
                    #print(np.shape(Gnew_1_bias))

                    #correction of 2nd momentum bias:
                    Gnew_2_bias= Gnew_2/(1.0-rho_2**epoch+1)        

                    # Hadamard product
                    update = np.c_[gamma*Gnew_1_bias/(delta+np.sqrt(np.diagonal(Gnew_2_bias)))]

                    beta_Ridge -= update

            y_Ridge_SGD_adam = X @ beta_Ridge
            
            y_Ridge_SGD_adam_array[l, g]= y_Ridge_SGD_adam

            MSE_Ridge_SGD_adam[l, g]=MSE(y_noisy, y_Ridge_SGD_adam)    

    #finding the minimum value of the MSE
    MSE_Ridge_SGD_adam_optimal = np.min(MSE_Ridge_SGD_adam)
    conditon = (MSE_Ridge_SGD_adam == MSE_Ridge_SGD_adam_optimal)
    #the l, g for which we have the minimum MSE
    result = np.where(conditon)
    print(result)

    print('method MSE=', MSE_Ridge_SGD_adam_optimal)  

    #saving the y that gives the optimal MSE
    y_Ridge_SGD_adam_optimal= y_Ridge_SGD_adam_array[result]
    y_Ridge_SGD_adam_optimal=(y_Ridge_SGD_adam_optimal[0])

    pd.DataFrame(MSE_Ridge_SGD_adam)
    mse_data_ridge_sgd_adam = pd.DataFrame(MSE_Ridge_SGD_adam)

    plt.figure(figsize=(12,7))
    sns.heatmap(data=mse_data_ridge_sgd_adam, annot=True,  fmt=".1e", cmap="crest")
    plt.xlabel("γ")
    plt.ylabel("λ")
    plt.show()

    plt.figure()
    plt.plot(x*180/np.pi, y_noisy, 'ro', label='data')
    plt.plot(x*180/np.pi, y_Ridge_SGD_adam_optimal, label='ADAM SGD Ridge')
    plt.xlabel('latitude [degrees]')
    plt.ylabel('f')
    plt.legend()
    plt.show()
    
#Calculating the time processing time
import time
start_time = time.time()
print("execution time=", (time.time() - start_time), 's')